# Topic: Team detection based on image embeddings

In [1]:
import os
import plotly.express as px

from ultralytics import YOLO
from PIL import Image


#### Step 1: load image data & trained model

In [2]:
#load images
img_files = os.listdir('../data/processed/ultimate_detect/images/train/')
imgs = [ Image.open('../data/processed/ultimate_detect/images/train/' + img_file) 
        for img_file in img_files]

#load YOLOv8 model

model = YOLO('./model/best.pt')

#### Step 2: Extract players using object detection model

In [3]:
img = imgs[1]
#detect players and extract subimages
results = model.predict(img, conf=0.5)

player_img = []
for result in results:
    for cls, box in zip(result.boxes.cls.tolist(), result.boxes.xyxy.tolist()):
        if cls == 0:
            cropped_img = img.crop(box)
            player_img.append(cropped_img)
            
#show extracted players

import plotly.subplots as sp
import plotly.express as px

# Assuming 'images' is your list of images
fig = sp.make_subplots(rows=1, cols=len(player_img))

for i, img in enumerate(player_img, start=1):
    fig.add_trace(
        px.imshow(img).data[0],
        row=1, col=i
    )
    
fig.update_layout(title_text="Extracted Players in original size")
fig.show()


0: 2176x3840 14 Players, 1 Referee, 130.2ms
Speed: 43.6ms preprocess, 130.2ms inference, 236.4ms postprocess per image at shape (1, 3, 2176, 3840)


# Calculate embeddings

In [4]:
from img2vec import Img2Vec

img2vec = Img2Vec()
embeddings = [img2vec.get_vec(img) for img in player_img]
#true_labels = [0,0,0,0,1,0,0,1,1,0,1,1,1,1]

c:\Users\marcc\miniconda3\envs\ultimate\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\marcc\miniconda3\envs\ultimate\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.



# K - NearestNeighbor clustering

In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)

pred_labels = kmeans.fit_predict(embeddings)

pred_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1])

# Visulize embeddings using PCA

In [6]:
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
import plotly.express as px


pca = PCA(n_components=3 )
embeddings_pca = pca.fit_transform(np.array(embeddings))

# Convert the PCA-reduced embeddings to a DataFrame
df = pd.DataFrame(embeddings_pca, columns=['Component 1', 'Component 2', 'Component 3'])

# Plot the PCA-reduced embeddings
fig = px.scatter_3d(df, x='Component 1', y='Component 2', z='Component 3', color=pred_labels)
fig.show()

In [7]:
import plotly.subplots as sp
import plotly.express as px

# Assuming 'images' is your list of images
fig = sp.make_subplots(rows=1, cols=len(player_img))

for i, img in enumerate(player_img, start=1):
    fig.add_trace(
        px.imshow(img.resize((224, 224))).data[0],
        row=1, col=i
    )
    fig.update_xaxes(title_text=str(pred_labels[i-1]), row=1, col=i)
    
fig.update_layout(title_text="Predicted classes for resized images")
fig.show()